In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Aniridia"
cohort = "GSE137996"

# Input paths
in_trait_dir = "../../input/GEO/Aniridia"
in_cohort_dir = "../../input/GEO/Aniridia/GSE137996"

# Output paths
out_data_file = "../../output/preprocess/Aniridia/GSE137996.csv"
out_gene_data_file = "../../output/preprocess/Aniridia/gene_data/GSE137996.csv"
out_clinical_data_file = "../../output/preprocess/Aniridia/clinical_data/GSE137996.csv"
json_path = "../../output/preprocess/Aniridia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Conjunctival mRNA and miRNA expression profiles in congenital aniridia are genotype and phenotype dependent (AKK mRNA)"
!Series_summary	"Purpose:"
!Series_summary	"To evaluate conjunctival cell microRNA and mRNA expression in relation to observed phenotype and genotype of aniridia-associated keratopathy (AAK) in a cohort of subjects with congenital aniridia."
!Series_summary	"Methods:"
!Series_summary	"Using impression cytology, bulbar conjunctival cells were sampled from 20 subjects with congenital aniridia and 20 age and sex-matched healthy control subjects. RNA was extracted and microRNA and mRNA analysis was performed using microarrays. Results were related to the presence and severity of AAK determined by a standardized clinical grading scale and to the genotype (PAX6 mutation?) determined by clinical genetics."
!Series_summary	"Results:"
!Series_summary	"Of the 2549 microRNAs analyzed, 21 were differentially expressed relative to controls. A

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains mRNA expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Aniridia/AAK): From key 2 and 3 we can determine if person has AAK and severity
trait_row = 2  # 'disease: AAK' or 'disease: healthy control'

# For age: Available at key 0
age_row = 0  # Age information is available

# For gender: Available at key 1
gender_row = 1  # Gender information is available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert the trait information to binary (0: control, 1: AAK)."""
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if 'disease:' in value:
        value = value.split('disease:')[-1].strip()
    
    if 'healthy control' in value or 'control' in value:
        return 0
    elif 'aak' in value or 'aniridia' in value:
        return 1
    else:
        return None

def convert_age(value):
    """Extract and convert age to continuous value."""
    if not isinstance(value, str):
        return None
    
    if 'age:' in value:
        try:
            age = int(value.split('age:')[-1].strip())
            return age
        except:
            return None
    return None

def convert_gender(value):
    """Convert gender to binary (0: female, 1: male)."""
    if not isinstance(value, str):
        return None
    
    value = value.lower().strip()
    if 'gender:' in value:
        value = value.split('gender:')[-1].strip()
    
    if value == 'f' or value == 'w':  # Assuming 'W' means woman
        return 0
    elif value == 'm':
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort, 
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Since we don't have the clinical_data.csv file, we need to create a clinical_df
    # The expected format for geo_select_clinical_features is a DataFrame where each row
    # represents a feature type and each column represents a sample
    
    # Create dummy data for demonstration purposes - the actual function will 
    # expect data in the proper format
    sample_ids = [f"GSM{i}" for i in range(1, 21)]  # 20 samples based on background info
    
    # Create empty DataFrame with sample IDs as columns
    clinical_df = pd.DataFrame(index=range(5), columns=sample_ids)
    
    # This is a placeholder to satisfy the function call
    # In a real scenario, we would need actual clinical data arranged properly
    
    # Since we don't have the actual clinical data, we should skip this part
    # and just acknowledge that clinical data extraction cannot be performed
    print("Cannot extract clinical features: clinical data file not found.")
    
    # Write an empty DataFrame to the output file to maintain workflow
    empty_clinical_df = pd.DataFrame(columns=['Sample', 'Aniridia', 'Age', 'Gender'])
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    empty_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Empty clinical data saved to {out_clinical_data_file}")


Cannot extract clinical features: clinical data file not found.
Empty clinical data saved to ../../output/preprocess/Aniridia/clinical_data/GSE137996.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502',
       'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529',
       'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581',
       'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315625',
       'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647'],
      dtype='object', name='ID')

Gene data dimensions: 58201 genes × 40 samples


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be Agilent microarray probe IDs (starting with "A_19_P"),
# not standard human gene symbols. These will need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns with probe IDs and gene symbols
# From the output, we can see:
# - Probe IDs in gene expression data are like 'A_19_P00315452'
# - In gene annotation, 'ID' contains probe IDs and 'GENE_SYMBOL' contains gene symbols

# 2. Get gene mapping dataframe by extracting the ID and GENE_SYMBOL columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply gene mapping to convert probe-level expression to gene expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the mapping process
print(f"\nOriginal probe data dimensions: {len(gene_data.index)} probes × {gene_data.shape[1]} samples")
print(f"Gene mapping dataframe dimensions: {gene_mapping.shape[0]} rows × {gene_mapping.shape[1]} columns")
print(f"Mapped gene data dimensions: {len(gene_data.index)} genes × {gene_data.shape[1]} samples")

# Preview the first few genes after mapping
print("\nFirst 10 gene symbols after mapping:")
print(list(gene_data.index[:10]))



Original probe data dimensions: 29222 probes × 40 samples
Gene mapping dataframe dimensions: 48862 rows × 2 columns
Mapped gene data dimensions: 29222 genes × 40 samples

First 10 gene symbols after mapping:
['A1BG', 'A1BG-AS1', 'A1CF', 'A1CF-2', 'A1CF-3', 'A2M', 'A2M-1', 'A2M-AS1', 'A2ML1', 'A2MP1']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols in the gene expression data...")
# From the previous step output, we can see the data already contains gene symbols
# like 'A1BG', 'A1CF', 'A2M' which need to be normalized
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene data shape: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print(f"Normalized gene data shape: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Extract clinical features from scratch instead of loading the empty file
print("Extracting clinical features from original clinical data...")
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait, 
    trait_row,
    convert_trait,
    age_row,
    convert_age,
    gender_row,
    convert_gender
)

# Save the extracted clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

print("Clinical features preview:")
print(preview_df(clinical_features))

# Check if clinical features were successfully extracted
if clinical_features.empty:
    print("Failed to extract clinical features. Dataset cannot be processed further.")
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note="Clinical features could not be extracted from the dataset."
    )
    print("Dataset deemed not usable due to lack of clinical features.")
else:
    # 2. Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")

    # 3. Handle missing values systematically
    linked_data = handle_missing_values(linked_data, trait_col=trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")

    # 4. Check if the dataset is biased
    print("\nChecking for bias in feature variables:")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

    # 5. Conduct final quality validation
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data for aniridia patients and healthy controls."
    )

    # 6. Save linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols in the gene expression data...
Original gene data shape: 29222 genes × 40 samples
Normalized gene data shape: 20778 genes × 40 samples


Normalized gene expression data saved to ../../output/preprocess/Aniridia/gene_data/GSE137996.csv
Extracting clinical features from original clinical data...
Clinical features saved to ../../output/preprocess/Aniridia/clinical_data/GSE137996.csv
Clinical features preview:
{'GSM4096389': [1.0, 20.0, 0.0], 'GSM4096390': [1.0, 20.0, 0.0], 'GSM4096391': [1.0, 28.0, 0.0], 'GSM4096392': [1.0, 20.0, 0.0], 'GSM4096393': [1.0, 38.0, 0.0], 'GSM4096394': [1.0, 57.0, 1.0], 'GSM4096395': [1.0, 26.0, 0.0], 'GSM4096396': [1.0, 18.0, 1.0], 'GSM4096397': [1.0, 36.0, 0.0], 'GSM4096398': [1.0, 42.0, 0.0], 'GSM4096399': [1.0, 18.0, 0.0], 'GSM4096400': [1.0, 42.0, 0.0], 'GSM4096401': [1.0, 36.0, 1.0], 'GSM4096402': [1.0, 28.0, 0.0], 'GSM4096403': [1.0, 55.0, 0.0], 'GSM4096404': [1.0, 54.0, 1.0], 'GSM4096405': [1.0, 34.0, 1.0], 'GSM4096406': [1.0, 51.0, 0.0], 'GSM4096407': [1.0, 46.0, 0.0], 'GSM4096408': [1.0, 52.0, 0.0], 'GSM4096409': [0.0, 53.0, 0.0], 'GSM4096410': [0.0, 54.0, 1.0], 'GSM4096411': [0.0, 40

Data shape after handling missing values: (40, 20781)

Checking for bias in feature variables:
For the feature 'Aniridia', the least common label is '1.0' with 20 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Aniridia' in this dataset is fine.

Quartiles for 'Age':
  25%: 25.75
  50% (Median): 34.0
  75%: 47.25
Min: 18.0
Max: 59.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 10 occurrences. This represents 25.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Aniridia/cohort_info.json


Linked data saved to ../../output/preprocess/Aniridia/GSE137996.csv
